<a href="https://colab.research.google.com/github/josemanuelvinhas/MarvelRecomverse/blob/main/MarvelRecomverse_API_de_Marvel_con_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. API de Marvel Comics**

La API de Marvel Comics es una herramienta para ayudar a los desarrolladores de todo el mundo a crear sitios web y aplicaciones utilizando datos de los más de 70 años de la era de los cómics Marvel. Se trata de un servicio RESTful que proporciona métodos para acceder a recursos específicos meidante URL. Permite buscar y filtrar conjuntos de recursos según varios criterios. Los resultados se devuelven en formato JSON.

Para hacer uso de la API es necesario registrarse, y de esta manera obtener una API Key. Esto se puede hacer desde [Marvel Developer Portal](https://developer.marvel.com/).

Una vez que se obtiene la API Key se puede probar la API mediante la herramienta [Interactive API Tester](https://developer.marvel.com/docs).

# **2. MarvelRecomverse: Uso de la API con python**

La aplicación a desarrollar necesita un dataset de personajes de cómics de Marvel y sus descripciones. Para crear este dataset se hará uso de la API de Marvel usando python.

Para obtener la información necesaria las peticiones deberán tener las siguientes características:

- Tipo: GET
- URL: https://gateway.marvel.com/v1/public/characters
- Parámetros: 
  1. *limit* : es el número máximo de personajes que devuelve la consulta (máx. 100)
  2. *offset* : debido al límite de personajes que devuelve la consulta, es necesario indicar a partir de qúe personaje queremos que la consulta nos devuelva resultados. Por ejemplo, si existen 300 personajes y queremos obtener los que van del 200 al 300 debemos indicar un *limit=100* y un *offset=200*
  3. *ts* : sello de tiempo
  4. *apikey* : la clave pública que proporciona Marvel
  5. *hash* : un hash de *Sello de Tiempo + Clave Privada + Clave Pública*

**1. Librerías:** se emplearán 4 librerías
- *request*: para realizar peticiones 
- *datetime*: para obtener el Sello de Tiempo
- *hashlib*: para realizar el hash requerido
- *pandas*: para manejar el dataset


In [2]:
import requests
import datetime
import hashlib
import pandas

**2. Construyendo la petición**
- *Sello de Tiempo*: se obtiene al comienzo del script mediante la librería *datetime*
- *API Key*: se proporcionan 2 claves, una publica, que se enviará como parámetro, y otra privada, que se emplea en el Hash.
- *Hash*: se obtendrá el *hash* usando el algoritmo md5 y obteniendo el resumen en hexadecimal 

In [3]:
timestamp = datetime.datetime.now().strftime('%Y-%m-%d%H:%M:%S')
public_api_key = 'd5d4682141673489b856445431f75246'
private_api_key = '8687323dce2867e5aab980b8dde10cb7e6f3d360'

hash_md5 = hashlib.md5()
hash_md5.update(f'{timestamp}{private_api_key}{public_api_key}'.encode('utf-8'))
hash = hash_md5.hexdigest()

- *limit*: como el objetivo es obtener todos los personajes este parámetro será el máximo posible (100)
- *offset*: como el objetivo es obtener todos los personajes este parámetro comenzará siendo 0 e irá avanzando de 100 en 100 hasta llegar a 1500, ya que hay un poco más de 1400 personajes.

**3. Realizando as consultas**

Se desea guardar el nombre y la descripción de cada personaje, para el cual se usarán 2 diccionarios. Se recorrerán los resultados obtenidos en cada petición y los personajes cuya descripción no sea vacía se guardarán.

Se empleará la librería *requests* para realizar la petición get con los parámetros deseados.

In [4]:
name = []
description = []

for i in range(0, 1500, 100):
    parametros = {'limit': 100, 'offset': i, 'ts': timestamp, 'apikey': public_api_key, 'hash': hash}
    print("Consulta a realizar con los siguientes parametros: ",parametros)
    respuesta = requests.get('https://gateway.marvel.com:443/v1/public/characters', parametros)
    resultados = respuesta.json()
    for personaje in resultados['data']['results']:
        if (personaje['description'].strip() != ""):
            name.append(personaje['name'])
            description.append(personaje['description'])

Consulta a realizar con los siguientes parametros:  {'limit': 100, 'offset': 0, 'ts': '2020-11-1911:52:33', 'apikey': 'd5d4682141673489b856445431f75246', 'hash': 'e48403e73c808a1e1ab59986201a7170'}
Consulta a realizar con los siguientes parametros:  {'limit': 100, 'offset': 100, 'ts': '2020-11-1911:52:33', 'apikey': 'd5d4682141673489b856445431f75246', 'hash': 'e48403e73c808a1e1ab59986201a7170'}
Consulta a realizar con los siguientes parametros:  {'limit': 100, 'offset': 200, 'ts': '2020-11-1911:52:33', 'apikey': 'd5d4682141673489b856445431f75246', 'hash': 'e48403e73c808a1e1ab59986201a7170'}
Consulta a realizar con los siguientes parametros:  {'limit': 100, 'offset': 300, 'ts': '2020-11-1911:52:33', 'apikey': 'd5d4682141673489b856445431f75246', 'hash': 'e48403e73c808a1e1ab59986201a7170'}
Consulta a realizar con los siguientes parametros:  {'limit': 100, 'offset': 400, 'ts': '2020-11-1911:52:33', 'apikey': 'd5d4682141673489b856445431f75246', 'hash': 'e48403e73c808a1e1ab59986201a7170'}
Co

**4. Almacenando los resultados**

Una vez se obtienen los resultados se guardarán en un documento *.csv* haciendo uso de la librería *pandas*.

*Nota: se guardarán los datos sin almacenar el índice, ya que se va a hacer uso del archivo usando pandas, que ya añade el índice*

In [13]:
data = {'name': name, 'description': description}
dataframe = pandas.DataFrame(data)
dataframe.to_csv("marvel.csv", index=False)

#**3. Preprocesado del dataset**

Para poder trabajar con los datos se deberá realizar un preprocesado de las descripciones.

El dataset con el que se trabaja contiene una colección de 281 personajes. Cada uno tiene dos campos:

1.   *name*: nombre del personaje
2.   *description*: descripcion del personaje

Se hará uso de la librería *pandas* para el preprocesado. A continuación se carga el csv con los datos


In [14]:
import pandas as pd

originalData = pd.read_csv('marvel.csv')
originalData

,name,description
0,A-Bomb (HAS),Rick Jones has been Hulk's best bud since day ...
1,A.I.M.,AIM is a terrorist organization bent on destro...
2,Abomination (Emil Blonsky),"Formerly known as Emil Blonsky, a spy of Sovie..."
3,Adam Warlock,Adam Warlock is an artificially created human ...
4,Agent X (Nijo),Originally a partner of the mind-altering assa...
...,...,...
276,Zarek,Zarek is a member of the Kree race with no sup...
277,Zodiak,"Twelve demons merged with Norman Harrison, who..."
278,Zombie (Simon Garth),War hero Simon Garth was turned into a zombie ...
279,Zuras,Zuras was once the leader of the Eternals.


A continuación se llevará a cabo el preprocesado, en la que se usarán 3 técnicas:

1.   ***Tokenization***: división del texto en palabras 
2.   Eliminación de ***stopwords***
3. ***Stemmization***: permite la obtención de la raíz de cada palabra sin que el resultado sea una palabra real.

In [9]:
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import nltk
nltk.download('punkt')
nltk.download('stopwords')

ps = PorterStemmer()

preprocessedText = []

for row in originalData.itertuples():
    text = word_tokenize(row[2]) ## indice de la columna que contiene la descripcion
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [ps.stem(w) for w in text if not w in stops and w.isalnum()]
    text = " ".join(text)
    
    preprocessedText.append(text)

preprocessedData = originalData
preprocessedData['processed_text'] = preprocessedText

preprocessedData

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,number,name,description,processed_text
0,281,A-Bomb (HAS),Rick Jones has been Hulk's best bud since day ...,ha
1,281,A.I.M.,AIM is a terrorist organization bent on destro...,
2,281,Abomination (Emil Blonsky),"Formerly known as Emil Blonsky, a spy of Sovie...",abomin emil blonski
3,281,Adam Warlock,Adam Warlock is an artificially created human ...,adam warlock
4,281,Agent X (Nijo),Originally a partner of the mind-altering assa...,agent X nijo
...,...,...,...,...
276,281,Zarek,Zarek is a member of the Kree race with no sup...,zarek
277,281,Zodiak,"Twelve demons merged with Norman Harrison, who...",zodiak
278,281,Zombie (Simon Garth),War hero Simon Garth was turned into a zombie ...,zombi simon garth
279,281,Zuras,Zuras was once the leader of the Eternals.,zura


#**4. Creación de la bolsa de palabras (BoW) con TF-IDF**

Se parte de los datos almacenados en "preprocessedData", en donde para cada personaje existe un campo 'preprocessed_text' que contiene la descripción preprocesada.

El objetivo es transformar todos los textos de sinopsis en vectores de frecuencias (Bag of words), aplicando además la ponderación TF-IDF para los valores de dichas frecuencias.

El paquete sklearn ofrece una clase llamada *TfidfVectorizer* que crea automáticamente la matriz compuesta por todos los vectores de frecuencias ponderados a partir de un array de textos (preprocessedData['processed_text'])

Si se quiere emplear la bolsa de palabras sin ponderación TF-IDF puede usarse la clase *CountVectorized* del mismo paquete

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

bagOfWordsModel = TfidfVectorizer()
bagOfWordsModel.fit(preprocessedData['processed_text'])
textsBoW= bagOfWordsModel.transform(preprocessedData['processed_text'])
print("Finished")

Finished


In [ ]:
textsBoW.shape

(281, 2283)

In [ ]:
print(textsBoW)

  (0, 2137)	0.130889104582123
  (0, 2058)	0.14217279372193212
  (0, 2004)	0.22205823661448335
  (0, 1974)	0.22205823661448335
  (0, 1894)	0.1961840975559547
  (0, 1790)	0.1961840975559547
  (0, 1779)	0.1752944840330487
  (0, 1634)	0.22205823661448335
  (0, 1505)	0.10190245579244313
  (0, 1404)	0.11629865466340347
  (0, 1174)	0.16198034620346324
  (0, 1086)	0.22205823661448335
  (0, 968)	0.1468449451157794
  (0, 840)	0.20692283552679946
  (0, 816)	0.17030995849742606
  (0, 797)	0.14942034497452006
  (0, 695)	0.1752944840330487
  (0, 647)	0.16198034620346324
  (0, 537)	0.1878544852619919
  (0, 496)	0.1878544852619919
  (0, 475)	0.22205823661448335
  (0, 284)	0.20692283552679946
  (0, 251)	0.22205823661448335
  (0, 242)	0.1878544852619919
  (0, 214)	0.1752944840330487
  :	:
  (278, 818)	0.30506443106319997
  (278, 81)	0.2225292018497309
  (279, 2282)	0.6554139206650713
  (279, 1148)	0.49150479114816825
  (279, 669)	0.5734593559066198
  (280, 1993)	0.22286926032113236
  (280, 1726)	0.25226

In [ ]:
bagOfWordsModel.get_feature_names()

In [ ]:
bagOfWordsModel.get_feature_names()[1005]

'incid'

#**5. Búsqueda de los personajes más similares a una persona**

Para realizar este proceso hay que realizar todo el proceso anterior incluyendo a la persona con su descripción.



In [15]:
nombre = "Pepe" #Introducir nombre
descripcion = "Pepe was once the leader of the Eternals. He is strong as iron and as small as an ant. He is a terrorist" #Descripcion

In [16]:
import pandas as pd

originalData = pd.read_csv('marvel.csv')
originalData = originalData.append({'name' : nombre, 'description' : descripcion}, ignore_index=True)
originalData

,name,description
0,A-Bomb (HAS),Rick Jones has been Hulk's best bud since day ...
1,A.I.M.,AIM is a terrorist organization bent on destro...
2,Abomination (Emil Blonsky),"Formerly known as Emil Blonsky, a spy of Sovie..."
3,Adam Warlock,Adam Warlock is an artificially created human ...
4,Agent X (Nijo),Originally a partner of the mind-altering assa...
...,...,...
277,Zodiak,"Twelve demons merged with Norman Harrison, who..."
278,Zombie (Simon Garth),War hero Simon Garth was turned into a zombie ...
279,Zuras,Zuras was once the leader of the Eternals.
280,Zzzax,"A chain reaction in an atomic reactor, a resul..."


In [17]:
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import nltk
nltk.download('punkt')
nltk.download('stopwords')

ps = PorterStemmer()

preprocessedText = []

for row in originalData.itertuples():
    text = word_tokenize(row[2]) ## indice de la columna que contiene la descripcion
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [ps.stem(w) for w in text if not w in stops and w.isalnum()]
    text = " ".join(text)
    
    preprocessedText.append(text)

preprocessedData = originalData
preprocessedData['processed_text'] = preprocessedText

preprocessedData

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,name,description,processed_text
0,A-Bomb (HAS),Rick Jones has been Hulk's best bud since day ...,rick jone hulk best bud sinc day one friend te...
1,A.I.M.,AIM is a terrorist organization bent on destro...,aim terrorist organ bent destroy world
2,Abomination (Emil Blonsky),"Formerly known as Emil Blonsky, a spy of Sovie...",formerli known emil blonski spi soviet yugosla...
3,Adam Warlock,Adam Warlock is an artificially created human ...,adam warlock artifici creat human born cocoon ...
4,Agent X (Nijo),Originally a partner of the mind-altering assa...,origin partner assassin black swan nijo spi de...
...,...,...,...
277,Zodiak,"Twelve demons merged with Norman Harrison, who...",twelv demon merg norman harrison soon adopt gu...
278,Zombie (Simon Garth),War hero Simon Garth was turned into a zombie ...,war hero simon garth turn zombi secretari layl...
279,Zuras,Zuras was once the leader of the Eternals.,zura leader etern
280,Zzzax,"A chain reaction in an atomic reactor, a resul...",A chain reaction atom reactor result terrorist...


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

bagOfWordsModel = TfidfVectorizer()
bagOfWordsModel.fit(preprocessedData['processed_text'])
textsBoW= bagOfWordsModel.transform(preprocessedData['processed_text'])

A continuación se crea una matriz N x N (N= número de personajes) en donde el valor de la posición matriz[ i , j ] indique la distancia que existe entre el personaje i y el personaje j. 

In [19]:
from sklearn.metrics import pairwise_distances

distance_matrix= pairwise_distances(textsBoW,textsBoW ,metric='cosine')


Ahora basta con obtener el índice de la persona introducida y las distancias con el resto de personajes

In [20]:
indexOfTitle = preprocessedData[preprocessedData['name']==nombre].index.values[0]
indexOfTitle

distance_scores = list(enumerate(distance_matrix[indexOfTitle]))
distance_scores

[(0, 0.9429037594051554),
 (1, 0.8737492680194615),
 (2, 1.0),
 (3, 1.0),
 (4, 1.0),
 (5, 0.9566567344748318),
 (6, 1.0),
 (7, 1.0),
 (8, 0.9481970831793141),
 (9, 1.0),
 (10, 0.9421206203190599),
 (11, 1.0),
 (12, 1.0),
 (13, 1.0),
 (14, 1.0),
 (15, 1.0),
 (16, 1.0),
 (17, 1.0),
 (18, 1.0),
 (19, 0.8838856415313219),
 (20, 1.0),
 (21, 1.0),
 (22, 1.0),
 (23, 1.0),
 (24, 1.0),
 (25, 1.0),
 (26, 1.0),
 (27, 1.0),
 (28, 1.0),
 (29, 1.0),
 (30, 1.0),
 (31, 1.0),
 (32, 1.0),
 (33, 1.0),
 (34, 1.0),
 (35, 1.0),
 (36, 1.0),
 (37, 1.0),
 (38, 0.9299958674731036),
 (39, 1.0),
 (40, 0.9478446467461085),
 (41, 0.9549177211043653),
 (42, 0.9189019686961419),
 (43, 0.9223008745731874),
 (44, 1.0),
 (45, 1.0),
 (46, 1.0),
 (47, 0.9511172137781917),
 (48, 1.0),
 (49, 1.0),
 (50, 0.9514025527164229),
 (51, 1.0),
 (52, 1.0),
 (53, 1.0),
 (54, 1.0),
 (55, 1.0),
 (56, 1.0),
 (57, 0.9497235166279826),
 (58, 0.9343482964519906),
 (59, 1.0),
 (60, 1.0),
 (61, 1.0),
 (62, 1.0),
 (63, 1.0),
 (64, 1.0),
 (65,

Una vez tenemos las distancias, únicamente debemos ordenarlas, quedarnos con las más cortas y mostrarlas de forma legible

In [21]:
ordered_scores = sorted(distance_scores, key=lambda x: x[1])
top_scores = ordered_scores[1:11]
top_indexes = [i[0] for i in top_scores]
preprocessedData['name'].iloc[top_indexes]

279                                 Zuras
171                                 Thena
170                      The Leader (HAS)
176                            Thor (MAA)
1                                  A.I.M.
19     Captain America/Steve Rogers (MAA)
71                            Shadow King
126                                Sprite
42              Iron Man/Tony Stark (MAA)
177                       Thor (Ultimate)
Name: name, dtype: object

**Búsqueda basada en Descripción + Valoración Comentarios + Valoración Directa (Likes) + Valoración Media Comentarios**

Ponderación:

* Descripción (70%): se basa en el parecido entre la descripción que tenga el usuario y la descripción de cada personaje
* Valoración Comentarios (7.5%): se basa en la valoración del comentario textual que realice el usuario a un personaje. Estos valores son extraídos del subsistema de valoración. Puede ser:
  * Buena (0)
  * Neutra o no valoración (0.5)
  * Mala (1)
* Valoración Directa por Likes (15%)
  * Like (0)
  * Dislike (1)
  * Nnguna (0.5)
* Valoración Media Comentarios (7.5%): se trata de la media de las valoraciones de comentarios de todos los usuarios sobre un personaje. Puede ser un valor entre 0 y 1.


Para probar esta parte del sistema simularemos tener los datos de las valoraciones de los comentarios y las valoraciones directas.

A continuación se generan aleatoriamente las valoraciones:

* 0  es valoración positiva
* 0.5 es valoración neutra (o no valoración)
* 1 es valoración negativa

In [50]:
from random import choice, randint

valoracion_comentario = []
valoracion_directa = []
valoracion_media = []

for i in range(0,len(preprocessedData)):
  valoracion_comentario.append(choice([0.0, 0.5, 1.0]))
  valoracion_directa.append(choice([0.0, 0.5, 1.0]))
  valoracion_media.append(randint(0.0,100.0)/100.0)

preprocessedData['valoracion_comentario'] = valoracion_comentario
preprocessedData['valoracion_directa'] = valoracion_directa
preprocessedData['valoracion_media'] = valoracion_media

preprocessedData

,name,description,processed_text,valoracion_comentario,valoracion_directa,valoracion_media
0,A-Bomb (HAS),Rick Jones has been Hulk's best bud since day ...,rick jone hulk best bud sinc day one friend te...,0.0,0.5,0.05
1,A.I.M.,AIM is a terrorist organization bent on destro...,aim terrorist organ bent destroy world,1.0,1.0,0.17
2,Abomination (Emil Blonsky),"Formerly known as Emil Blonsky, a spy of Sovie...",formerli known emil blonski spi soviet yugosla...,0.5,0.0,0.92
3,Adam Warlock,Adam Warlock is an artificially created human ...,adam warlock artifici creat human born cocoon ...,0.5,0.5,0.87
4,Agent X (Nijo),Originally a partner of the mind-altering assa...,origin partner assassin black swan nijo spi de...,0.0,1.0,0.54
...,...,...,...,...,...,...
277,Zodiak,"Twelve demons merged with Norman Harrison, who...",twelv demon merg norman harrison soon adopt gu...,0.5,1.0,0.41
278,Zombie (Simon Garth),War hero Simon Garth was turned into a zombie ...,war hero simon garth turn zombi secretari layl...,0.0,0.5,0.02
279,Zuras,Zuras was once the leader of the Eternals.,zura leader etern,1.0,1.0,0.96
280,Zzzax,"A chain reaction in an atomic reactor, a resul...",A chain reaction atom reactor result terrorist...,0.5,0.0,0.95


A continuación calculamos las distancias teniendo en cuenta la ponderación

In [51]:
distance_scores_final = []

for i in range(len(distance_matrix[indexOfTitle])):
  valoracion = distance_matrix[indexOfTitle][i]*0.7 + preprocessedData['valoracion_comentario'][i]*0.075 + preprocessedData['valoracion_directa'][i]*0.15 + preprocessedData['valoracion_media'][i]*0.075
  distance_scores_final.append((i,valoracion))

distance_scores_final

[(0, 0.7387826315836087),
 (1, 0.849374487613623),
 (2, 0.8065),
 (3, 0.8777499999999999),
 (4, 0.8905),
 (5, 0.7911597141323822),
 (6, 0.9504999999999999),
 (7, 0.7397499999999999),
 (8, 0.9224879582255199),
 (9, 0.8109999999999999),
 (10, 0.8529844342233419),
 (11, 0.8949999999999999),
 (12, 0.9564999999999999),
 (13, 0.91525),
 (14, 0.9167499999999998),
 (15, 0.8709999999999999),
 (16, 0.7772499999999999),
 (17, 0.8229999999999998),
 (18, 0.7427499999999999),
 (19, 0.6967199490719252),
 (20, 0.8679999999999999),
 (21, 0.90625),
 (22, 0.90625),
 (23, 0.76825),
 (24, 0.7922499999999999),
 (25, 0.7929999999999999),
 (26, 0.8567499999999999),
 (27, 0.94),
 (28, 0.8994999999999999),
 (29, 0.72475),
 (30, 0.85975),
 (31, 0.8724999999999998),
 (32, 0.7404999999999999),
 (33, 0.8469999999999999),
 (34, 0.7705),
 (35, 0.8334999999999999),
 (36, 0.868),
 (37, 0.835),
 (38, 0.8197471072311724),
 (39, 0.9279999999999999),
 (40, 0.9154912527222759),
 (41, 0.6706924047730557),
 (42, 0.86448137808

A continuación nos quedamos con los personajes más aprecidos

In [52]:
ordered_scores_final = sorted(distance_scores_final, key=lambda x: x[1])
top_scores_final = ordered_scores_final[1:11]
top_indexes_final = [i[0] for i in top_scores_final]
preprocessedData['name'].iloc[top_indexes_final]

176                            Thor (MAA)
171                                 Thena
41                               Iron Man
220                                 Vapor
71                            Shadow King
153                  Supreme Intelligence
43            Iron Patriot (James Rhodes)
19     Captain America/Steve Rogers (MAA)
234                                Vision
115                            Spider-Man
Name: name, dtype: object

Si comparamos estas lista con la que se obtuvo previamente se puede apreciar como las ponderaciones afectaron al resultado final

In [48]:
preprocessedData['name'].iloc[top_indexes]

279                                 Zuras
171                                 Thena
170                      The Leader (HAS)
176                            Thor (MAA)
1                                  A.I.M.
19     Captain America/Steve Rogers (MAA)
71                            Shadow King
126                                Sprite
42              Iron Man/Tony Stark (MAA)
177                       Thor (Ultimate)
Name: name, dtype: object

#**6. Añadir, Modificar y Borrar un personaje del dataset**

Para **añadir** un personaje al final del DataFrame

1.   Se cargan los datos del csv
2.   Se añade el nuevo personaje con el método *append()*
3.   Se guardan los datos

Añadir a *Mr. Yomiquesh*:


In [ ]:
import pandas as pd

originalData = pd.read_csv('marvel.csv')
originalData = originalData.append({'name' : 'Mr. Yomiquesh', 'description' : 'Yomiquesh was once the leader of the Eternals. He is strong as iron and as small as an ant. He is a terrorist'}, ignore_index=True)
originalData.to_csv("marvel_add.csv", index=False)

originalData

,name,description
0,A-Bomb (HAS),Rick Jones has been Hulk's best bud since day ...
1,A.I.M.,AIM is a terrorist organization bent on destro...
2,Abomination (Emil Blonsky),"Formerly known as Emil Blonsky, a spy of Sovie..."
3,Adam Warlock,Adam Warlock is an artificially created human ...
4,Agent X (Nijo),Originally a partner of the mind-altering assa...
...,...,...
277,Zodiak,"Twelve demons merged with Norman Harrison, who..."
278,Zombie (Simon Garth),War hero Simon Garth was turned into a zombie ...
279,Zuras,Zuras was once the leader of the Eternals.
280,Zzzax,"A chain reaction in an atomic reactor, a resul..."


Para **modificar** un personaje al final del DataFrame

1.   Se cargan los datos del csv
3.   Se modifica el personaje
3.   Se guardan los datos

Modificar la descripción de *Mr. Yomiquesh*:

In [ ]:
import pandas as pd

originalData = pd.read_csv('marvel_add.csv')
originalData.loc[originalData['name'] == 'Mr. Yomiquesh', 'description'] = 'Yomiquesh was once the leader of the Eternals.'
originalData.to_csv("marvel_mod.csv", index=False)

originalData

,name,description
0,A-Bomb (HAS),Rick Jones has been Hulk's best bud since day ...
1,A.I.M.,AIM is a terrorist organization bent on destro...
2,Abomination (Emil Blonsky),"Formerly known as Emil Blonsky, a spy of Sovie..."
3,Adam Warlock,Adam Warlock is an artificially created human ...
4,Agent X (Nijo),Originally a partner of the mind-altering assa...
...,...,...
277,Zodiak,"Twelve demons merged with Norman Harrison, who..."
278,Zombie (Simon Garth),War hero Simon Garth was turned into a zombie ...
279,Zuras,Zuras was once the leader of the Eternals.
280,Zzzax,"A chain reaction in an atomic reactor, a resul..."


Para **borrar** un personaje al final del DataFrame

1.   Se cargan los datos del csv
2.   Se obtiene el índice del personaje
3.   Se borra el personaje
3.   Se guardan los datos

Eliminar a *Mr. Yomiquesh*:

In [ ]:
import pandas as pd

originalData = pd.read_csv('marvel_add.csv')
indexOfCharacter = originalData[originalData['name']=="Mr. Yomiquesh"].index.values[0]
originalData = originalData.drop(indexOfCharacter, axis=0)
originalData.to_csv("marvel_delete.csv", index=False)

originalData

,name,description
0,A-Bomb (HAS),Rick Jones has been Hulk's best bud since day ...
1,A.I.M.,AIM is a terrorist organization bent on destro...
2,Abomination (Emil Blonsky),"Formerly known as Emil Blonsky, a spy of Sovie..."
3,Adam Warlock,Adam Warlock is an artificially created human ...
4,Agent X (Nijo),Originally a partner of the mind-altering assa...
...,...,...
276,Zarek,Zarek is a member of the Kree race with no sup...
277,Zodiak,"Twelve demons merged with Norman Harrison, who..."
278,Zombie (Simon Garth),War hero Simon Garth was turned into a zombie ...
279,Zuras,Zuras was once the leader of the Eternals.
